# Análisis de programas de gobierno Perú
- Limpieza y estructuración de una base de datos útil.
- Información descriptiva.
- Modelo de tópicos.

In [47]:
# Espacio de trabajo
import tqdm
import numpy as np
from numpy.random import seed
from numpy.random import randint
#import lda2vec as l2v
!pip install PyPDF2
import PyPDF2
import gensim
from gensim import corpora, similarities, models
from gensim.models.phrases import Phraser
#from gensim.models.wrappers import LdaMallet
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
%matplotlib inline
try:
    import seaborn
    from spacy_spanish_lemmatizer import SpacyCustomLemmatizer
except:
    pass
import sys
import pandas as pd
import spacy
import string
import wordcloud
import nltk
import os
nltk.download('stopwords')
from nltk.corpus import stopwords
from pprint import pprint
import re
import logging
import pyLDAvis
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator
pyLDAvis.enable_notebook()
import pyLDAvis.gensim_models

C:\Users\usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Pre-procesamiento de los datos

In [66]:
path = r'C:\Users\usuario\Desktop\Programas de Gobierno'
programas = list()
for i in os.listdir(path):
    if i.endswith(".pdf"):
        programa = PyPDF2.PdfReader(path + "/" + i)
        number_of_pages = len(programa.pages)
        for i in range(number_of_pages):
            text = programa.pages[i].extract_text()
            programas.append(text)
programas

C:\Users\usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['  \n PLAN DE GOBIERNO REGIÓN AREQUIPA 2023 -2026 – ROHEL SÁNCHEZ SÁNCHEZ  \n \n89 Ilustración 1. Megatendencias gl obales.  \n \n                                           Fuente : CEPLAN. Observatorio de prospectiva . \n \nEnvejecimiento de la población.  \nA nivel mundial, la densidad poblacional y la estructura etaria apuntan a cambios sin precedentes \nen las nuevas formas de vida de la población. El envejecimiento de la población seguirá teniendo \nun crecimiento sostenido en el tiempo producto del aumento de la esperanza de vida al nacer y \nde la reducción de la tasa de natalidad.  El mundo estará compuesto por una gran cantidad de \npersonas envejecidas; pese a que, en el periodo 2019 -2021, con la COVID -19, se incrementó la \ntasa de mortalidad y morbilidad en la población de 65 años a más a nivel mundial.  Según el último \ncenso (2017), l a población del grupo de 65 años  a más  se incre mentó en las últimas dos décadas , \n7,4% en el 2007 y 8,8% en el 2017.  \n \nRápida 

In [83]:
# Limpieza
programas=[re.sub(r"/(\r\n|\n|\r)/gm", "", x) for x in programas]
programas=[re.sub(r'([^0-9a-zA-Z])\1+', r'\1', x) for x in programas]
programas=[re.sub(r'^[^0-9a-zA-Z@#]+', '', x) for x in programas]
programas=[re.sub(r'()\1+', r'\1\1', x) for x in programas]
programas=[re.sub(r'(.)\1+', r'\1\1', x) for x in programas]
programas=[re.sub(r'([A-Z a-z]+)\1+', r'\1\1', x) for x in programas]
programas=[re.sub(r"(?<=[a-z])\r?\n"," ", x) for x in programas]
programas=[re.sub(r"\n", " ", x) for x in programas]
programas=[re.sub(r'[0-9]+', '', x) for x in programas]
programas=[re.sub(r'[^\w\s]', '', x) for x in programas]
programas=[re.sub(r'(?:^| )\w(?:$| )', '', x) for x in programas]
programas=[x.lower() for x in programas]
spanish_stopwords=stopwords.words('spanish')
spanish_stopwords.extend(['las','los','si','sí','de','lo','la','una'])
spanish_stopwords.extend(list(string.ascii_lowercase))
programas = [word for word in programas if not word in spanish_stopwords]
programas=[x.strip() for x in programas]
programas=[re.sub(' +', ' ', x) for x in programas]
programas

C:\Users\usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['plan de gobierno región arequipa rohel sánchez sánchez ilustración megatendencias gl obales fuente ceplan observatorio de prospectiva envejecimiento de la población nivel mundial la densidad poblacionalla estructura etaria apuntancambios sin precedentes en las nuevas formas de vida de la población el envejecimiento de la población seguirá teniendo un crecimiento sostenido en el tiempo producto del aumento de la esperanza de vida al nacer de la reducción de la tasa de natalidad el mundo estará compuesto por una gran cantidad de personas envejecidas peseque en el periodo con la covid se incrementó la tasa de mortalidadmorbilidad en la población de añosmásnivel mundial según el último censopoblación del grupo de añosmás se incre mentó en las últimas dos décadas en el en el rápida urbanización global en el futuro la urbanizaciónnivel mundial será parte de los cambios sin precedentes en las nuevas formas de vida de la población habrá un crecimiento exp ansivo respectola urbanización globa

In [84]:
programas_migracion = list()
for i in range(len(programas)):
    text = programas[i]
    expresion = re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,10}migra(\w+)(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,10}", text)
    try:
        programas_migracion.append(expresion.group())
    except:
        try:
            expresion_2 = re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,10}extran(\w+)(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,10}", text)
            programas_migracion.append(expresion_2.group())
        except:
            try:
                expresion_3 = re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,10}refugia(\w+)(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,10}", text)
                programas_migracion.append(expresion_3.group())
            except:
                try:
                    expresion_4 = re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,10}venez(\w+)(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,10}", text)
                    programas_migracion.append(expresion_4.group())
                except:
                    programas_migracion.append('No match')
programas_migracion

C:\Users\usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['No match',
 'las preferencias de la poblaciónprobablemente muchos de ellos decidan migrar zonas alejadas de la ciudada zonas suburbanas grandes asequibles creci',
 'graciasque desarrolla una economía urbana del sector terciario las migraciones las invasionesla concentración demográfica en las zonas urbanas',
 'ambos años obtendremos un total de nuevos estudiantes que migraron ese año de las instituciones particulareslas instituciones públicas',
 'extranjeros residentes en el país en especial de los j',
 'correspondecasos de dengue considerados graves sin embargoon el proceso de migración existente en el perú poblaciones de la región amaz',
 'demuestra que chiclayo se ha ido por el proceso de migración en una metrópoli de más de habitantes en',
 'No match',
 'No match',
 'de seguridad los colegioprofesionales los taxistas los comerciantes ambulanteslos ciudadanos extranjeros que viven en las zonas de lima metropolitana con cada',
 'indiferenciapoco involucramiento en la lucha contra l

In [86]:
programas_migracion = [x for x in programas_migracion if not 'No match' in x]
programas_migracion

C:\Users\usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['las preferencias de la poblaciónprobablemente muchos de ellos decidan migrar zonas alejadas de la ciudada zonas suburbanas grandes asequibles creci',
 'graciasque desarrolla una economía urbana del sector terciario las migraciones las invasionesla concentración demográfica en las zonas urbanas',
 'ambos años obtendremos un total de nuevos estudiantes que migraron ese año de las instituciones particulareslas instituciones públicas',
 'extranjeros residentes en el país en especial de los j',
 'correspondecasos de dengue considerados graves sin embargoon el proceso de migración existente en el perú poblaciones de la región amaz',
 'demuestra que chiclayo se ha ido por el proceso de migración en una metrópoli de más de habitantes en',
 'de seguridad los colegioprofesionales los taxistas los comerciantes ambulanteslos ciudadanos extranjeros que viven en las zonas de lima metropolitana con cada',
 'indiferenciapoco involucramiento en la lucha contra la delincuencia de ciudadanos extranjero

In [89]:
with open('programas_migracion.txt', 'w') as f:
    for item in programas_migracion:
        # write each item on a new line
        f.write("%s\n," % item)
    print('Done')

Done


C:\Users\usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [93]:
with open('programas_migracion.txt') as f:
    lines = f.read().splitlines()
lines

C:\Users\usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['las preferencias de la población probablemente muchos de ellos decidan migrar zonas alejadas de la ciudada zonas suburbanas grandes asequibles crecimiento',
 'gracias que desarrolla una economía urbana del sector terciario las migraciones las invasiones la concentración demográfica en las zonas urbanas',
 'ambos años obtendremos un total de nuevos estudiantes que migraron ese año de las instituciones particulares las instituciones públicas',
 'extranjeros residentes en el país en especial de los',
 'corresponde casos de dengue considerados graves sin embargo el proceso de migración existente en el perú poblaciones de la región amazonía',
 'demuestra que chiclayo se ha ido por el proceso de migración en una metrópoli de más de habitantes en',
 'de seguridad los colegio profesionales los taxistas los comerciantes ambulantes los ciudadanos extranjeros que viven en las zonas de lima metropolitana con cada',
 'indiferencia poco involucramiento en la lucha contra la delincuencia de ciudada

In [96]:
programas_migracion = list()
stopwords = ['el', 'la', 'las', 'de', 'los', 'cual', 'se', 'ha', 'en', 'un','que']
for item in lines:
    querywords = item.split()
    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    result = ' '.join(resultwords)
    programas_migracion.append(result)
programas_migracion

C:\Users\usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['preferencias población probablemente muchos ellos decidan migrar zonas alejadas ciudada zonas suburbanas grandes asequibles crecimiento',
 'gracias desarrolla una economía urbana del sector terciario migraciones invasiones concentración demográfica zonas urbanas',
 'ambos años obtendremos total nuevos estudiantes migraron ese año instituciones particulares instituciones públicas',
 'extranjeros residentes país especial',
 'corresponde casos dengue considerados graves sin embargo proceso migración existente perú poblaciones región amazonía',
 'demuestra chiclayo ido por proceso migración una metrópoli más habitantes',
 'seguridad colegio profesionales taxistas comerciantes ambulantes ciudadanos extranjeros viven zonas lima metropolitana con cada',
 'indiferencia poco involucramiento lucha contra delincuencia ciudadanos extranjeros residentes lima integrarlos ciudadanos extranjeros radican lima',
 'extranjeros necesitamos involucramiento todos vecinos esa manera',
 'últimos años produj

In [98]:
with open('programas_migracion_clean.txt', 'w') as f:
    for item in programas_migracion:
        # write each item on a new line
        f.write("%s\n" % item)
    print('Done')

Done


C:\Users\usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [99]:
with open('programas_migracion_clean.txt') as f:
    programas_migracion = f.read().splitlines()
programas_migracion

C:\Users\usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['preferencias población probablemente muchos ellos decidan migrar zonas alejadas ciudada zonas suburbanas grandes asequibles crecimiento',
 'gracias desarrolla una economía urbana del sector terciario migraciones invasiones concentración demográfica zonas urbanas',
 'ambos años obtendremos total nuevos estudiantes migraron ese año instituciones particulares instituciones públicas',
 'extranjeros residentes país especial',
 'corresponde casos dengue considerados graves sin embargo proceso migración existente perú poblaciones región amazonía',
 'demuestra chiclayo ido por proceso migración una metrópoli más habitantes',
 'seguridad colegio profesionales taxistas comerciantes ambulantes ciudadanos extranjeros viven zonas lima metropolitana con cada',
 'indiferencia poco involucramiento lucha contra delincuencia ciudadanos extranjeros residentes lima integrarlos ciudadanos extranjeros radican lima',
 'extranjeros necesitamos involucramiento todos vecinos esa manera',
 'últimos años produj

In [97]:
# Le decimos a spaCy que vamos con toda
#spacy.prefer_gpu()
#spacy.require_gpu()
# Corremos los datos en el core de español
#nlp = spacy.load("es_core_news_sm", exclude=["ner", "parser"])
nlp = spacy.load("es_dep_news_trf", exclude=["ner", "parser"])
#lemmatizer_spa = SpacyCustomLemmatizer()
#nlp.add_pipe("spanish_lemmatizer", name = 'lemmatizer', after = 'attribute_ruler')
#nlp.replace_pipe("lemmatizer", "spanish_lemmatizer")
#Vemos cuales son los componentes del pipeline
print(nlp.pipe_names)
#Aplicamos el pipe a cada objeto de nuestra lista de tweets
print('---------SPACY-----------')
%time docs = list(nlp.pipe(programas_migracion))

C:\Users\usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


OSError: [E050] Can't find model 'es_dep_news_trf'. It doesn't seem to be a Python package or a valid path to a data directory.